In [1]:
from CompositeBeam import CompositeBeams

![compositebeamexample.png](\Resources\CompositeBeamExample.png)

In [2]:
cp = CompositeBeams(DeadLoad   = 3.73, #kN/m^2
                    LiveLoad_c = 1   , #kN/m^2

                    # Kompozit çalışma aşaması yükleri
                    SuperDeadLoad = 0.0, #kN/m^2
                    LiveLoad      = 2.0, #kN/m^2
                    w_girder      = 1.0, #kN/m kiriş ağırlığı

                    # Beton bilgisi
                    f_ck     = 30  ,   #N/mm2
                    wc       = 2400,   #kg/m3

                    # Çelik malzeme bilgisi
                    Fy       = 355, #N/mm2
                    Fu_beam  = 355*1.2, #N/mm2

                    # Çelik enkesit özellikleri
                    Abeam     = 33_420,     #mm^2
                    Hbeam     = 800 ,       #mm
                    Hweb      = 734  ,    #mm
                    b_flange  = 300  ,      #mm
                    t_flange  = 33   ,     #mm
                    t_web     = 17.5 ,      #mm
                    Ix        = 2790.8*10**4, #mm^4
                    Ycon      = 150  ,     #mm Çelik kesit üst başlığından en üst beton lifine olan mesafe

                    # Kiriş uzunluk ve aralık bilgisi
                    Laralık  = 4000,  #mm
                    Lbeam    = 10000,  #mm

                    # Stud çivisi bilgisi
                    D_stud               = 19       , #mm
                    H_stud               = 100      ,  #mm
                    b_studhead           = 32      ,   #mm
                    t_studhead           = 10      ,  #mm
                    Fy_stud              = 235     ,   #N/mm2 çelik ankrajın minimum(karakteristik) akma dayanımı
                    Fu_stud              = 235*1.2 ,   #N/mm2 çelik ankrajın minimum(karakteristik) çekme dayanımı
                    CompRatio            = 1,    #Kompozitlik oranı minimum %25 olabilir. %100 tam etkileşimli kompozit demektir.
                    Nw                   = 1,       #Adet Kesit başlığının genişliğinde atılan stud çivisi adeti
                    x                    = 30 ,     #mm Kesit başlığının genişliğinde atılan stud çivileri arasındaki mesafe
                    IsWebAlignmentWelded = False ,  #Stud çivisi kiriş gövdesi izasında mı kaynaklanıyor.

                    # Metal sac bilgisi
                    hr           = 50,    #mm
                    wr           = 100,   #mm
                    t_sac        = 1.0,   #mm
                    RibsDistance = 250,   #mm 2*wr
                    IsParallel   = True
    )

In [3]:
GeomCheck = cp.ConcAvailableStressCheck(f_ck = cp.f_ck)
GeomCheck = cp.SteelAvailableStressCheck(Fy = cp.Fy)
if GeomCheck != True:
    print("Geometrik kontroller sağlanmalı hesap yapılmadı!!!\n")
# Kiriş enkesit kontrolü
SectionCheck = cp.SectionCheck(Hbeam=cp.Hbeam, t_web=cp.t_web, Fy=cp.Fy)
if SectionCheck != True:
    print("Kesit değiştirin!!!\n")

20 N/mm^2 ≤ 30 N/mm^2 < 70 N/mm^2 √ - TSSDC 12.2.3(a)

355 N/mm^2 ≤ 460 N/mm^2 √ - TSSDC 12.2.3(c)

h/tw = 45.71 ≤ 3.76*(E/Fy)^0.5 = 89.25 √



In [8]:
b_eff       = cp.EffectiveSlabWidth(L = cp.Lbeam, Lu = cp.Laralık) # Effektif döşeme genişliği
tc          = cp.Calc_tc(hr = cp.hr,Ycon = cp.Ycon, IsParallel = cp.IsParallel) # Stud çivisiz beton yüksekliği
Ac          = cp.Calc_Ac(b_eff=b_eff, tc=tc) # Stud çivisiz beton alanı
Cc          = cp.CrushConcCapacity(fck = cp.f_ck,Ac=Ac)
Cs          = cp.YieldBeamCapacity(fy = cp.Fy, As = cp.Abeam)
Cf          = cp.Calc_Cf(Vbeam=Cs, Vcon=Cc)

b_eff = 2500.0mm

tc = 125.0mm
Ac = 312500.0mm^2


In [9]:
a           = cp.Calc_a(Cf =Cf, fck = cp.f_ck, b_eff = b_eff)
Y2          = cp.Calc_Y2(Ycon = cp.Ycon, a = a)
Y1          = cp.Calc_Y1(T_steel=Cs, C_conc=Cc, b_flange=cp.b_flange, t_flange = cp.t_flange, t_web = cp.t_web,BeamFy = cp.Fy)
C_flange    = cp.Calc_Cflange(t_flange = cp.t_flange, Y1 = Y1, b_flange = cp.b_flange, BeamFy = cp.Fy)
C_web       = cp.Calc_Cweb(Y1=Y1, t_flange = cp.t_flange, t_web = cp.t_web, BeamFy = cp.Fy)
Mn_web      = cp.PTEInWebMn(T_steel=Cs, C_conc=Cc, Cflange=C_flange, Cweb=C_web, Hbeam = cp.Hbeam, t_flange =cp.t_flange, Y1=Y1, Y2=Y2)
Mn_flange   = cp.PTEInFlangeMn(C_conc=Cc, Cflange=C_flange, Hbeam = cp.Hbeam, BeamAs = cp.Abeam, BeamFy = cp.Fy, Y1=Y1, Y2=Y2)
Mn_slab     = cp.PTEInSlabMn(Cf=Cf, Hbeam = cp.Hbeam, hr = cp.hr, tc=tc, a=a)
Mn_design   = cp.CompositeBeamDesignFlexuralCapacity2(C_conc=Cc, T_steel=Cs,Mn_web=Mn_web, Mn_slab=Mn_slab,Mn_flange=Mn_flange, Y1=Y1, t_flange = cp.t_flange, fi_b=0.9)

a = 125.0mm

Y2 = 87.5mm

PTE başlıkta kabul edilirse Y1 = (T_steel - C_conc) / (2*BeamFy*b_flange) = 18.29mm
Kompozit kiriş kısmi etkileşimlidir. Kompozitlik oranı %25'in altına inmemelidir.

Y1 = 18.29 < 33 = t_flange PTE kiriş başlığındadır.

φMn = φ * Mn_flange = 0.9 * 5407.286kNm = 4866.558kNm



![Example12_7_answer.png](\Resources\Example12_7_answer.png)